# Author: Markus Laubenthal
This Notebook uses the time embedding module.

In [1]:
!wget "https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip" --no-verbose
!unzip spatio_merged_data_iss.zip
!rm spatio_merged_data_iss.zip

2021-01-31 17:30:00 URL:https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip [37282886/37282886] -> "spatio_merged_data_iss.zip" [1]
Archive:  spatio_merged_data_iss.zip
   creating: input/
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/grid_ML.geojson   
  inflating: __MACOSX/input/._grid_ML.geojson  
  inflating: input/internet_ML.csv   
  inflating: __MACOSX/input/._internet_ML.csv  
  inflating: input/satelite.png      
  inflating: __MACOSX/input/._satelite.png  
  inflating: input/weather.csv       
  inflating: __MACOSX/input/._weather.csv  
  inflating: input/social_pulse_ML.csv  
  inflating: __MACOSX/input/._social_pulse_ML.csv  


In [2]:
!rm -rf functions
!git clone https://github.com/markuslaubenthal/lab_st.git functions
!cd functions && git checkout original_densenet

Cloning into 'functions'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 574 (delta 178), reused 157 (delta 83), pack-reused 314
Receiving objects: 100% (574/574), 893.58 KiB | 22.91 MiB/s, done.
Resolving deltas: 100% (375/375), done.
Branch 'original_densenet' set up to track remote branch 'original_densenet' from 'origin'.
Switched to a new branch 'original_densenet'


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, activations
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file
from keras.optimizers import SGD, Adam


In [4]:
from functions.preprocessing.DataImport import load_and_scale_internet, load_and_scale_satelite, load_and_scale_social, load_and_scale_weather, create_space_invariant
from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file

In [5]:
f = getFileHandler('training_data.h5')
internet, internet_origin, internet_min, internet_max = load_and_scale_internet('input/internet_ML.csv', f, log10=False, min_max_axis=1)


In [6]:
x_closeness = generate_dataset(internet, [1,2,3], 168, f, "closeness")
x_period = generate_dataset(internet, [24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 167, 168], 168, f, "period")
y = generate_label(internet, 168, f)



In [7]:
date_range = pd.date_range('2013-10-31 23:00:00', '2014-01-01 22:00:00', freq='h')
#Hour of Day, Day of Week, Holiday
time_one_hot = np.zeros((1488, 24 + 7 + 1))
for i, date in enumerate(date_range):
  time_one_hot[i, date.hour] = 1
  time_one_hot[i, 24 + date.dayofweek] = 1
  day = i // 24
  if day in [0,54,56,61,62]:
    time_one_hot[i,24 + 7] = 1

In [8]:
from functions.preprocessing.TestTrainSplit import seven_days_train_test_split

x_closeness_train, x_closeness_test = seven_days_train_test_split(x_closeness, 168)
x_period_train, x_period_test = seven_days_train_test_split(x_period, 168)
y_train, y_test = seven_days_train_test_split(y, 168)

In [9]:
from functions.model.SplitDenseNetFactory import SplitDenseNetFactory
from keras.optimizers import SGD, Adam
dn = SplitDenseNetFactory()
model = dn.Model((x_closeness_train.shape[3], x_period_train.shape[3]), depth=8, growth_rate=8, initial_filters=8, time_embedding_method="scale_0_2", t_minus_one=False)

In [12]:
def scheduler(epoch, lr):
  return lr / 10 ** (epoch // 10)


lr = 0.0001
epochs = 16
model.compile(optimizer=Adam(lr=lr, decay= lr/float(epochs)),
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()],
              )


model.fit([x_closeness_train, x_period_train, time_one_hot[168:-168]], y_train,
          validation_data=([x_closeness_test, x_period_test, time_one_hot[-168:]], y_test), epochs=epochs
          )

Epoch 1/16
36/36 [==============================] - 8s 187ms/step - loss: 0.0035 - root_mean_squared_error: 0.0591 - val_loss: 0.0027 - val_root_mean_squared_error: 0.0518
Epoch 2/16
36/36 [==============================] - 6s 176ms/step - loss: 0.0035 - root_mean_squared_error: 0.0590 - val_loss: 0.0027 - val_root_mean_squared_error: 0.0521
Epoch 3/16
36/36 [==============================] - 6s 176ms/step - loss: 0.0034 - root_mean_squared_error: 0.0585 - val_loss: 0.0027 - val_root_mean_squared_error: 0.0516
Epoch 4/16
36/36 [==============================] - 6s 176ms/step - loss: 0.0034 - root_mean_squared_error: 0.0580 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0513
Epoch 5/16
36/36 [==============================] - 6s 177ms/step - loss: 0.0034 - root_mean_squared_error: 0.0584 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0507
Epoch 6/16
36/36 [==============================] - 6s 177ms/step - loss: 0.0034 - root_mean_squared_error: 0.0581 - val_loss: 0.0027 - val_

In [ ]:
from functions.postprocessing.ErrorEvaluation import calculate_errors

In [ ]:
pred = model.predict([x_closeness[:], x_period[:], time_one_hot[168:]])

pred, base = calculate_errors(pred, internet_origin, internet_min, internet_max, log10=False)

all:  90.35721303183061
test:  92.4639425682171
val:  74.3193444015834
